In [14]:
import pandas as pd
import numpy as np
import random 
random.seed(123)
np.random.seed(123)

## Set pakdd root path & read data 

In [15]:
root_path = "./"
train_ = root_path+"train.csv"
dev_   = root_path+"validation.csv"
test_  = root_path+"test.csv"

train = pd.read_csv(train_, header=None, names=["abstract","label"])
dev   = pd.read_csv(dev_, header=None, names=["abstract","label"])
test  = pd.read_csv(test_, header=None, names=["abstract"])
X_train = train["abstract"].values
y_train = train["label"].values
X_dev = dev["abstract"].values
y_dev = dev["label"].values
X_test = test["abstract"].values
print(train.shape, dev.shape, test.shape)

(16800, 2) (11200, 2) (7000, 1)


In [16]:
label_dict = {'LO': 0, 'DC': 1, 'SE': 2, 'DS': 3, 'CL': 4, 'NI': 5, 'CR': 6}
ind2label = {0:'LO', 1:'DC', 2:'SE', 3:'DS', 4:'CL', 5:'NI', 6:'CR'}
labels = list(label_dict)
print("-----------train-------------")
for l in labels:
    print(l, sum(np.array(y_train)==l)/len(y_train))
print("-----------dev-------------")
for l in labels:
    print(l, sum(np.array(y_dev)==l)/len(y_dev))

# set indicies for splitting dev further into dev & test 
# from sklearn.model_selection import train_test_split
# X_dev, X_test, y_dev, y_test = train_test_split(X_dev, y_dev, test_size=0.50, random_state=1, stratify=y_dev)
# print("-----------dev-------------")
# for l in labels:
#     print(l, sum(np.array(y_dev)==l)/len(y_dev))
# print("-----------dev-------------")
# for l in labels:
#     print(l, sum(np.array(y_test)==l)/len(y_test))
    
dev_indices = np.load("dev_indices.npy")
test_indices = np.load("test_indices.npy")
    
X_dev[dev_indices]

-----------train-------------
LO 0.10779761904761904
DC 0.12154761904761904
SE 0.12178571428571429
DS 0.16291666666666665
CL 0.1630952380952381
NI 0.16452380952380952
CR 0.15833333333333333
-----------dev-------------
LO 0.10866071428571429
DC 0.12098214285714286
SE 0.11848214285714286
DS 0.15839285714285714
CL 0.16660714285714287
NI 0.16303571428571428
CR 0.16383928571428572


array(["  Model-based testing (MBT) is a well-known technology, which allows for\nautomatic test case generation, execution and evaluation. To test\nnon-functional properties, a number of test MBT frameworks have been developed\nto test systems with real-time, continuous behaviour, symbolic data and\nquantitative system aspects. Notably, a lot of these frameworks are based on\nTretmans' classical input/output conformance (ioco) framework. However, a\nmodel-based test theory handling probabilistic behaviour does not exist yet.\nProbability plays a role in many different systems: unreliable communication\nchannels, randomized algorithms and communication protocols, service level\nagreements pinning down up-time percentages, etc. Therefore, a probabilistic\ntest theory is of great practical importance. We present the ingredients for a\nprobabilistic variant of ioco and define the {\\pi}oco relation, show that it\nconservatively extends ioco and define the concepts of test case, execution 

In [4]:
# change to fasttext format

with open("pakdd2021.train", "w") as f:
    for i in range(len(y_train)):
        f.write("__label__{} {}\n".format(y_train[i], X_train[i].replace("\n","").lower()))
        
# internal valid
X_dev_prepro = list()
with open("pakdd2021.valid", "w") as f:
    for i in dev_indices:
        d_ = X_dev[i].replace("\n","").lower()
        f.write("__label__{} {}\n".format(y_dev[i], d_))
        X_dev_prepro.append(d_)
        
# internal test
X_test_prepro = list()
with open("pakdd2021.test", "w") as f:
    for i in test_indices:
        d_ = X_dev[i].replace("\n","").lower()
        f.write("__label__{} {}\n".format(y_dev[i], d_))
        X_test_prepro.append(d_)
        
# dev all
X_dev_prepro_all = list()
for i in range(len(y_dev)):
    d_ = X_dev[i].replace("\n","").lower()
    X_dev_prepro_all.append(d_)
    
# final test set released by organizers
X_finaltest_prepro = list()
with open("pakdd2021.finaltest", "w") as f:
    for i in range(7000):
        d_ = X_test[i].replace("\n","").lower()
        f.write("__label__{} {}\n".format(0, d_))
        X_finaltest_prepro.append(d_)

In [6]:
# train
import fasttext

model = fasttext.train_supervised(
    input='pakdd2021.train', 
    autotuneValidationFile='pakdd2021.valid',
#     autotuneDuration=600
)

In [7]:
# predict
# eval
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

# dev
y_pred = list()
y_prob = list()

# for d_ in X_dev_prepro:
#     label = model_pseudo.predict(d_)
# #     print(label)
#     y_prob.append(label[1][0])
#     y_pred.append(label[0][0].replace("__label__",""))
    
# dev
for d_ in X_dev_prepro_all:
    label = model.predict(d_)
#     print(label)
    y_prob.append(label[1][0])
    y_pred.append(label[0][0].replace("__label__",""))
    
print("dev weighted F1", f1_score(np.array(y_dev)[test_indices], 
                                  np.array(y_pred)[test_indices], average='weighted'))
print(confusion_matrix(np.array(y_dev)[test_indices], np.array(y_pred)[test_indices]))

np.save("y_dev_fasttext_pred_", np.array(y_pred), allow_pickle=False)
    
    
# test
y_pred = list()
y_prob = list()

for d_ in X_finaltest_prepro:
    label = model.predict(d_)
#     print(label)
    y_prob.append(label[1][0])
    y_pred.append(label[0][0].replace("__label__",""))
    
# print("test weighted F1", f1_score(np.array(y_dev)[test_indices], 
#                                   np.array(y_pred)[test_indices], average='weighted'))
# print(confusion_matrix(np.array(y_dev)[test_indices], np.array(y_pred)[test_indices]))
    
np.save("y_test_fasttext_pred", np.array(y_pred), allow_pickle=False)

dev weighted F1 0.8817286330133082
[[904   3   2   4   8   1  11]
 [ 10 784  25  18   8  48  25]
 [  8  29 514  54   4  41  28]
 [  5   8  36 821   8   5   4]
 [  9   6  12  13 548   0  20]
 [  6  34  47  11   1 803  11]
 [ 10  22  25   5  30   5 566]]


## TagMe Pipline

In [8]:
USERNAME = 'parklize'
TOKEN = '39040dc2-960a-4d32-a6e6-7761aa14c55f-843339462'

import tagme
import time
# Set the authorization token for subsequent calls.
tagme.GCUBE_TOKEN = TOKEN

In [9]:
def get_ann_str(text:str):
    """ Get TagMe annotation from string
    
    Returns
    ------------------
    entity string with index[tab]entity:score...[newline]
    """
    lunch_annotations = tagme.annotate(text)

    # Print annotations with a score higher than 0.1
    ann_str = ''
    for ann in lunch_annotations.get_annotations(0.1):
#         print(ann)
        ann_str += (ann.entity_title+':')
        ann_str += (str(ann.score)+'\t')
    ann_str += '\n'
    return ann_str

In [10]:
import sys
def get_ann(to_ann_list, filepath, start_index=0):
    size = len(to_ann_list)
    i = start_index
    print("Processing {}-th line".format(i))
    for t in to_ann_list[i:]:
        time.sleep(2) # sleep 1s
        try:
            t_ = to_ann_list[i].replace("\n","").lower()
            ann_str = get_ann_str(t_)
            with open(filepath, "a") as f:
                f.write('{}\t{}'.format(i, ann_str))
                i += 1
        except Exception as e:
            print(i, e)
            time.sleep(60) # sleep 60s and retry
            get_ann(to_ann_list, filepath, i)

In [21]:
# get_ann(X_train, filepath="X_train_TagMe_ann.txt")
# get_ann(X_dev, filepath="X_dev_TagMe_ann.txt")
get_ann(X_test, filepath="X_test_TagMe_ann.txt")

Processing 0-th line


### Run denoisying to get entity-based abstract embeddings

In [11]:
import re
import numpy as np
from sklearn.cluster import KMeans
from wikipedia2vec import Wikipedia2Vec

MODEL_FILE = '/home/parklize/Documents/Code/wikipedia2vec/enwiki_20180420_win10_100d.pkl'
wiki2vec = Wikipedia2Vec.load(MODEL_FILE)

In [12]:
def get_entity_dict(ann_str):
    """
    Parameters
    -----------------
    ann_str: extracted annotated entity string for each example of X_train/dev/test
    
    Returns
    -----------------
    dictionary version of it 
    """
    ent_dict = dict()
    splitted = ann_str.split("\t")
    ind = splitted[0]
    for s in splitted[1:]:
        if len(s) > 2:
#             print('----',s)
            match = re.match(r'(.*):(\d\.\d+)', s)
            ent_dict[match.group(1)] = match.group(2)
        
    return ent_dict

def get_embs(ent_dict):
    """
    Parameters
    ---------------
    ent_dict: entity:score dictionary
    
    Returns
    ---------------
    entity: embedding dictionary with Wikipedia2Vec
    """
    emb_dict = dict()
    for k in ent_dict:
        try:
            emb_dict[k] = wiki2vec.get_entity_vector(k)
        except Exception as e:
            print("exception for getting entity vector", k, e)
            emb_dict[k] = np.zeros(100)
    return emb_dict

In [22]:
# get embeddings

target = "test"
target_file = 'X_{}_TagMe_ann.txt'.format(target)

emb_list = list()
with open(target_file, 'r') as f:
    lines = f.readlines()
    for ind,l in enumerate(lines):
        ent_dict = get_entity_dict(l)
        emb_dict = get_embs(ent_dict) # for one example (line)
        datapoints = np.asarray(list(emb_dict.values()))
        print(ind, "--------", ent_dict)
        if datapoints.shape[0] > 2:
            kmeans = KMeans(n_clusters=2, random_state=0).fit(datapoints)

            # which label has more entities -> core label
            zero_labels = len(kmeans.labels_)-sum(kmeans.labels_)
            one_labels = sum(kmeans.labels_)
            core_label = 0 if zero_labels > one_labels else 1 

            print(zero_labels, one_labels, core_label)

            # select entities of the core label
            filtered_keys = np.array(list(emb_dict.keys()))[kmeans.labels_==core_label]
            filtered_emb_dict = {k: emb_dict[k] for k in filtered_keys}
            filtered_ent_dict = {k: ent_dict[k] for k in filtered_keys}
            
            print("-------- filtered keys")
            print(filtered_keys)

            # get abstract embedding
            emb = np.zeros(100)
            denominator = np.sum(np.exp(np.array(list(filtered_ent_dict.values()), dtype=np.float32)))
#             print(denominator)
            for k in filtered_keys:
                emb += (np.exp(float(filtered_ent_dict[k]))/denominator)*np.array(filtered_emb_dict[k], dtype=np.float32)
        else:
            emb = np.zeros(100)
        emb_list.append(emb)
        
        if len(emb_list) % 100 == 0:
            print(len(emb_list))
            
emb_np = np.array(emb_list)
print(emb_np.shape)

np.save("X_{}_ent_emb__".format(target),emb_np)

exception for getting entity vector ProActive 
0 -------- {'Analytical chemistry': '0.11855411529541016', 'Special relativity': '0.14349859952926636', 'Focus (optics)': '0.10620374977588654', 'Discrete event simulation': '0.42268040776252747', 'Simulation': '0.11282288283109665', 'Synchronization': '0.22753968834877014', 'Algorithm': '0.20013675093650818', 'Ad hoc': '0.12051313370466232', 'Grid computing': '0.176923006772995', 'Java (programming language)': '0.14870071411132812', 'Simulation language': '0.6711933612823486', 'GPSS': '0.19117647409439087', 'Shock (circulatory)': '0.14540867507457733', 'Antimicrobial resistance': '0.10702858120203018', 'Time': '0.19959656894207', 'Software framework': '0.18583542108535767', 'ProActive': '0.11334767937660217', 'Parallel computing': '0.2190283089876175', 'Drug resistance': '0.14785826206207275', 'Database transaction': '0.19921493530273438', 'Time (magazine)': '0.1014566421508789', 'Time travel': '0.18711018562316895', 'Paper': '0.195870518

7 9 1
-------- filtered keys
['Multi-party system' 'Broadcasting' 'Terrestrial television'
 'Television channel' 'Treaty' 'Veto' 'Voting' 'Anonymity'
 'Minority group']
12 -------- {'Mathematical logic': '0.36525964736938477', 'Calculus': '0.3173087239265442', 'Presentation of a group': '0.11362902820110321', 'Conservative extension': '0.5', 'Lazy evaluation': '0.5', 'Structure (mathematical logic)': '0.20076745748519897', 'Extension (semantics)': '0.1147153377532959'}
6 1 0
-------- filtered keys
['Mathematical logic' 'Presentation of a group' 'Conservative extension'
 'Lazy evaluation' 'Structure (mathematical logic)'
 'Extension (semantics)']
exception for getting entity vector Areto 
13 -------- {'Paper': '0.10637331753969193', 'Homogeneity and heterogeneity': '0.19668594002723694', 'Mathematical optimization': '0.42072951793670654', 'Number': '0.19000115990638733', 'Areto': '0.5', 'Quality of service': '0.2559245824813843', 'Bandwidth (signal processing)': '0.29991796612739563', '

11 16 1
-------- filtered keys
['Electrical efficiency' 'Broadcasting' 'Wireless sensor network'
 'Node (networking)' 'Wireless network' 'Computer network'
 'Information theory' 'Natural selection' 'Social network'
 'Efficiency (Network Science)' 'Packet switching'
 'Artificial neural network' 'Transmission (telecommunications)'
 'Information' 'Radio receiver' 'Linear network coding']
24 -------- {'Property (philosophy)': '0.2730894982814789', 'Notion (philosophy)': '0.253978967666626', 'Validity': '0.3172948956489563', 'Formula': '0.2715676426887512', 'Logic': '0.27051013708114624', 'Specification (technical standard)': '0.25993287563323975', 'Network packet': '0.2516081929206848', 'Sequence': '0.13832314312458038', 'Television program': '0.14685393869876862'}
2 7 1
-------- filtered keys
['Property (philosophy)' 'Notion (philosophy)' 'Validity' 'Logic'
 'Specification (technical standard)' 'Network packet' 'Sequence']
25 -------- {'Number': '0.185890793800354', 'Cryptography': '0.421

3 11 1
-------- filtered keys
['Conjecture' 'Church–Rosser theorem' 'Formal language'
 'Mathematical proof' 'Theory (mathematical logic)' 'Computation'
 'Graph theory' 'Automata theory' 'Lambda calculus' 'Turing machine'
 'Context-free language']
36 -------- {'Real-time Control System': '0.5', 'Control system': '0.21996068954467773', 'Software': '0.24114270508289337', 'Sparse matrix': '0.15515004098415375', 'Algorithm': '0.26149922609329224', 'Modernity': '0.1713898628950119', 'Software engineering': '0.645662784576416', 'Skill': '0.2224445343017578', 'Design': '0.2108568698167801', 'Unified Modeling Language': '0.3211970329284668', 'Industrial engineering': '0.2732057571411133', 'Technical standard': '0.23635822534561157', 'Academic publishing': '0.13709935545921326', 'British undergraduate degree classification': '0.11732283979654312', 'Analysis': '0.13532252609729767', 'Control engineering': '0.2390393167734146', 'Systems engineering': '0.49922698736190796', 'Technical support': '0.

47 -------- {'Integral': '0.12398681789636612', 'Rule-based system': '0.19735576212406158', 'Semantic Web': '0.4432234466075897', 'Hybrid vehicle': '0.13170021772384644', 'Knowledge': '0.15875904262065887', 'Base pair': '0.15677914023399353', 'Bus garage': '0.15980878472328186', 'Glossary of North American railway terms': '0.15641386806964874', 'Guarded logic': '0.5', 'Logic programming': '0.6270928382873535', 'Litre': '0.18766695261001587', 'Knowledge representation and reasoning': '0.14870987832546234', 'Description logic': '0.15909090638160706', 'Variable (mathematics)': '0.1813013106584549', 'Rule of inference': '0.14637809991836548', 'Positivism': '0.15546557307243347', 'Atom': '0.2281775176525116', 'Physical body': '0.21047113835811615', 'Decidability (logic)': '0.3561522960662842', 'Satisfiability': '0.23792777955532074', 'William Close': '0.12685254216194153', 'Web Ontology Language': '0.1666666716337204', 'Lambda calculus': '0.15052485466003418', 'Stable model semantics': '0.5

...

22 13 0
-------- filtered keys
['Computer network' 'Telecommunications network' 'Web traffic' 'Malware'
 'Denial-of-service attack' 'Attack (computing)'
 'Content-control software' 'Access control list' 'Router (computing)'
 'Content-addressable memory' 'Resource allocation'
 'Classless Inter-Domain Routing' 'Firewall (computing)'
 'September 11 attacks' 'Software design' 'Mathematical optimization'
 'Computational complexity theory' 'Algorithmic efficiency' 'Algorithm'
 'Data' 'Set (mathematics)' 'Tin can']
145 -------- {'Mathematical analysis': '0.12051480263471603', 'Software framework': '0.22420789301395416', 'Peer-to-peer': '0.34342846274375916', 'Weighted arithmetic mean': '0.37149161100387573', 'Download': '0.18817871809005737', 'Information': '0.1992833912372589', 'Digital distribution': '0.20446644723415375', 'Bandwidth (computing)': '0.195188507437706', 'Time': '0.14280188083648682', 'Mathematical optimization': '0.18535645306110382', 'Data transmission': '0.32019007205963135

21 1 0
-------- filtered keys
['Network topology' 'Broadcasting' 'Communication' 'Node (networking)'
 'Wireless sensor network' 'Predistribution' 'Shared secret' 'Sensor'
 'Method (computer programming)' 'Initialization (programming)'
 'Signaling (telecommunications)' 'Channel (communications)'
 'Computer network' 'Canadian dollar' 'Efficiency' 'Scalability' 'Number'
 'Vertex (geometry)' 'Usability testing' 'Usability' 'Algorithm']
exception for getting entity vector Fast Search &amp; Transfer 
157 -------- {'Loose coupling': '0.1866806447505951', 'Computer programming': '0.22368457913398743', 'Paradigm': '0.2086593359708786', 'Computer program': '0.3097081780433655', 'Scientific calculator': '0.13933800160884857', 'Software': '0.166046604514122', 'Petascale computing': '0.3829360902309418', 'Operating system': '0.24612678587436676', 'Substantial form': '0.16195301711559296', 'Data': '0.12687888741493225', 'Computer file': '0.23962460458278656', 'Email': '0.12687928974628448', 'Welfare

26 7 0
-------- filtered keys
['Mathematical logic' 'Markov decision process' 'Process philosophy'
 'Quality (philosophy)' 'Well-formed formula' 'Logic' 'Probability theory'
 'Bit' 'Probability' 'Present tense' 'Time complexity' 'Model checking'
 'Algorithm' 'Induced subgraph' 'Equivalence relation'
 'Material conditional' 'Set (mathematics)' 'Finite group'
 'Exterior algebra' 'Choice' 'Quantum state' 'Bisimulation'
 'Algorithmic efficiency' 'Qualitative property' 'Binary relation'
 'Finite set']
170 -------- {'Economic development': '0.12394804507493973', 'Maxima and minima': '0.12144165486097336', 'Degree of a polynomial': '0.21839216351509094', 'Bounded set': '0.18658295273780823', 'Linearity': '0.5651386976242065', 'Linear function': '0.2857784628868103', 'We (novel)': '0.10026036947965622', 'Present tense': '0.1291607767343521', 'Causality': '0.25819656252861023', 'Tree': '0.11781338602304459', 'Theorem': '0.11884589493274689', 'Construct (philosophy)': '0.10032214969396591'}
6 6 

14 14 1
-------- filtered keys
['Obfuscation (software)' 'Software' 'Software development'
 'Java (programming language)' 'Application software' 'Code refactoring'
 'Software development process' 'Decompiler' 'Computer file' 'Source code'
 'Bytecode' 'Method overriding' 'Constant (computer programming)'
 'Information hiding']
182 -------- {'Business Process Model and Notation': '0.5498194694519043', 'Industrial engineering': '0.1434241682291031', 'Standardization': '0.14014501869678497', 'Usability': '0.17983582615852356', 'Topic Maps': '0.2416369915008545', 'Mast (sailing)': '0.12302947789430618', 'Pattern': '0.15862004458904266', 'Business': '0.10189884901046753'}
4 4 1
-------- filtered keys
['Business Process Model and Notation' 'Standardization' 'Usability'
 'Topic Maps']
183 -------- {'Convention (norm)': '0.1375141143798828', 'Technology': '0.11246741563081741', 'Categorization': '0.1282244622707367', 'Selknam people': '0.12915661931037903', 'Evolution': '0.11689013242721558', '

KeyboardInterrupt: 